In [ ]:
# Import libraries
from cil.framework import AcquisitionGeometry 
from cil.plugins.astra.operators import ProjectionOperator
from cil.optimisation.algorithms import PDHG, CGLS
from cil.optimisation.operators import GradientOperator, BlockOperator
from cil.optimisation.functions import IndicatorBox, BlockFunction,L2NormSquared
from cil.io import NEXUSDataWriter, NEXUSDataReader
from cil.utilities.display import show2D

import numpy as np

In [ ]:
# Regularisation parameters for each case
alpha={'18':1e-05, '36':0.01, '72':0.018, '360':0.3}

In [ ]:
max_iterations = 100 # change to 3000 to match results of the paper

In [ ]:
for i in [18, 36, 72, 360]:  
    
    # load sparse data
    name_proj = "SparseData/data_{}".format(i)    
    reader = NEXUSDataReader(file_name=name_proj+".nxs")
    data = reader.load_data()
    
    # get acquisition geometry
    ag = data.geometry
    ig = ag.get_ImageGeometry()
    ig.voxel_num_x = 256
    ig.voxel_num_y = 256
    
    # Define ProjectionOperator, GradientOperator
    Aop = ProjectionOperator(ig, data.geometry, 'gpu')        
    L_operator = GradientOperator(ig, correlation = "SpaceChannels") 
    K = BlockOperator(Aop, L_operator)
    
    # Nonnegativity constraint    
    G = IndicatorBox(lower=0)
    
    normK = K.norm()
    sigma = 1./normK
    tau = 1./normK        

    F = BlockFunction(L2NormSquared(b=data), alpha[str(i)] * L2NormSquared())
    
    x_init = ig.allocate()
    pdhg = PDHG(initial = x_init, f = F, g = G, operator=K, max_iteration = max_iterations,
                update_objective_interval = 100)    
    pdhg.run(verbose=0)
    
    name_recon = "TikhonovReconstruction_projections_{}".format(i)
    writer = NEXUSDataWriter(file_name = "Tikhonov_reconstructions/"+name_recon+".nxs",
                         data = pdhg.solution)
    writer.write()      

    show2D(pdhg.solution, slice_list = [0,5,10,16], num_cols=4, origin="upper",
                       cmap="inferno", title="Projections {}".format(i), size=(25, 20))
    